## SSH Eddies

In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import datetime
import jdcal
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
ds = xr.open_dataset('/home/cz2397/data/ssh-eddies/tracks.20130125.nc', decode_cf=False)

In [3]:
df = ds.to_dataframe()

In [4]:
def jday_to_datetime(jday, refday=0):
    y, m, d, f = jdcal.jd2gcal(jday, refday)
    h = int(f*24)
    return pd.to_datetime(datetime.datetime(y, m, d, h))

In [5]:
pd_date = df.j1.apply(jday_to_datetime)

In [6]:
df.j1 = pd_date
df = df.rename(columns = {'j1': 'date'})

In [7]:
lon_fix_01 = df.where(df.lon <= 540).lon - 360
lon_fix_02 = df.where(df.lon > 540).lon - 720
lon_fix_A = lon_fix_01
lon_fix_A = lon_fix_A.fillna(lon_fix_02)
df.lon = lon_fix_A

In [8]:
lon_fix_03 = df.where(df.lon < 0).lon + 360
lon_fix_04 = df.where(df.lon >= 0).lon
lon_fix_B = lon_fix_03
lon_fix_B = lon_fix_B.fillna(lon_fix_04)
df.lon = lon_fix_B

In [9]:
track = np.asarray(df.track)
n = np.asarray(df.n)
date = df.date
lon = np.asarray(df.lon)
lat = np.asarray(df.lat)

In [10]:
M = max(track)
print(M)

215184


In [11]:
N = len(n)
print(N)

2590938


In [12]:
df.head()

,track,n,date,cyc,lon,lat,A,L,U
Nobs,,,,,,,,,
0,1,1,1992-10-14 12:00:00,-1,276.739990,-65.001099,1.24564,44.464001,5.97993
1,1,2,1992-10-21 12:00:00,-1,275.989990,-65.137604,2.64604,36.702400,8.17373
2,1,3,1992-10-28 12:00:00,-1,275.820007,-65.245697,3.40119,45.422100,8.28628
3,1,4,1992-11-04 12:00:00,-1,275.844971,-65.293297,1.45836,43.021999,5.20364
4,2,1,1992-10-14 12:00:00,-1,200.642029,-62.542400,2.16156,63.069698,4.81408


In [13]:
ssh_cchdo = MongoClient().eddies.ssh_cchdo

In [14]:
for i in tqdm(range(N)):
    eddy = {
        'eid': str(track[i]) + "-" + str(n[i]),
        'dat': date[i],
        'loc': [float(lon[i]), float(lat[i])]
    }
    ssh_cchdo.insert_one(dict(eddy))

100%|██████████| 2590938/2590938 [11:03<00:00, 3906.07it/s]
